In [1]:
import torch

In [2]:
import argparse
args = argparse.Namespace()
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.batch_size = 8
args.n_views = 2
args.temperature = 0.07
args.fp16_precision = False
args.arch = 'vgg'
args.log_every_n_steps = 100
args.learning_rate = 0.00003
args.epochs = 150
args.disable_cuda = not torch.cuda.is_available()
args.dataset_dir = r"E:\Data\ADNI\adni-fnirt-corrected"
args.csv_path = r"E:\Data\ADNI\pheno_ADNI_longitudinal_new.csv"
args.dataset_name = 'mri'
args.weight_decay = 1e-4
args.out_dim = 128
args

Namespace(device=device(type='cuda'), batch_size=8, n_views=2, temperature=0.07, fp16_precision=False, arch='resnet', log_every_n_steps=100, learning_rate=3e-05, epochs=150, disable_cuda=False, dataset_dir='E:\\Data\\ADNI\\adni-fnirt-corrected', csv_path='E:\\Data\\ADNI\\pheno_ADNI_longitudinal_new.csv', dataset_name='mri', weight_decay=0.0001, out_dim=128)

In [3]:
class BaseSimCLRException(Exception):
    """Base exception"""


class InvalidBackboneError(BaseSimCLRException):
    """Raised when the choice of backbone Convnet is invalid."""


class InvalidDatasetSelection(BaseSimCLRException):
    """Raised when the choice of dataset is invalid."""

In [4]:
# import numpy as np

# np.random.seed(0)


class ContrastiveLearningViewGenerator(object):
    """Take two random crops of one image as the query and key."""

    def __init__(self, base_transform, n_views=2):
        self.base_transform = base_transform
        self.n_views = n_views

    def __call__(self, x):
        return [self.base_transform(x) for i in range(self.n_views)]

In [5]:
from torchvision.transforms import transforms
from torchvision import transforms, datasets
from mri_dataset import ADNIDataset
from monai.transforms import *
from data_augmentation import MRIAugmentation

class ContrastiveLearningDataset:
    def __init__(self, root_folder, csv_path):
        self.root_folder = root_folder
        self.csv_path = csv_path

    @staticmethod
    def get_simclr_pipeline_transform(size, s=1):
        """Return a set of data augmentation transformations as described in the SimCLR paper."""
        # data_transforms = Compose([
        #     RandRotate90(prob=0.5, spatial_axes=[1, 2]),
        #     RandRotate90(prob=0.5, spatial_axes=[0, 1]),
        #     RandRotate90(prob=0.5, spatial_axes=[0, 2]),
        #     RandFlip(prob=0.5, spatial_axis=0),
        #     RandFlip(prob=0.5, spatial_axis=1),
        #     RandFlip(prob=0.5, spatial_axis=2),
            
        #     RandAdjustContrast(prob=1, gamma=(0.5, 1.5)),
        #     RandGaussianNoise(prob=1),
        #     # RandAffine(prob=args['prob'], translate_range=10, scale_range=(0.9, 1.1), rotate_range=45),
            
        #     Resize(spatial_size=[100, 100, 100]),
        #     NormalizeIntensity(channel_wise=True),
        # ])
        data_transforms = MRIAugmentation.get_augmentation_transforms()
    
        
        return data_transforms

    def get_dataset(self, name, n_views):
        valid_datasets = {}

        valid_datasets['mri'] = lambda: ADNIDataset(data_dir=self.root_folder, csv_path=self.csv_path, transform=ContrastiveLearningViewGenerator(self.get_simclr_pipeline_transform(100), n_views))
        try:
            dataset_fn = valid_datasets[name]
        except KeyError:
            raise InvalidDatasetSelection()
        else:
            return dataset_fn()

In [6]:
dataset = ContrastiveLearningDataset(args.dataset_dir, args.csv_path)
train_dataset = dataset.get_dataset(args.dataset_name, args.n_views)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True)

In [7]:
import os
import shutil

import torch
import yaml


def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def save_config_file(model_checkpoints_folder, args):
    if not os.path.exists(model_checkpoints_folder):
        os.makedirs(model_checkpoints_folder)
    with open(os.path.join(model_checkpoints_folder, 'config.yml'), 'w') as outfile:
        args_dict = vars(args)
        args_dict['device'] = args_dict['device'].type
        yaml.dump(args_dict, outfile, default_flow_style=False)


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [8]:
import logging
import os

import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# torch.manual_seed(0)


class SimCLR(object):

    def __init__(self, **kwargs):
        self.args = kwargs['args']
        self.model = kwargs['model'].to(self.args.device)
        self.optimizer = kwargs['optimizer']
        self.scheduler = kwargs['scheduler']
        self.writer = SummaryWriter()
        logging.basicConfig(filename=os.path.join(self.writer.log_dir, 'training.log'), level=logging.DEBUG)
        self.criterion = torch.nn.CrossEntropyLoss().to(self.args.device)

    def info_nce_loss(self, features):

        labels = torch.cat([torch.arange(self.args.batch_size) for i in range(self.args.n_views)], dim=0)
        labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
        labels = labels.to(self.args.device)

        features = F.normalize(features, dim=1)

        similarity_matrix = torch.matmul(features, features.T)
        # assert similarity_matrix.shape == (
        #     self.args.n_views * self.args.batch_size, self.args.n_views * self.args.batch_size)
        # assert similarity_matrix.shape == labels.shape

        # discard the main diagonal from both: labels and similarities matrix
        mask = torch.eye(labels.shape[0], dtype=torch.bool).to(self.args.device)
        labels = labels[~mask].view(labels.shape[0], -1)
        similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)
        # assert similarity_matrix.shape == labels.shape

        # select and combine multiple positives
        positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1)

        # select only the negatives the negatives
        negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1)

        logits = torch.cat([positives, negatives], dim=1)
        labels = torch.zeros(logits.shape[0], dtype=torch.long).to(self.args.device)

        logits = logits / self.args.temperature
        return logits, labels

    def train(self, train_loader):

        scaler = GradScaler(enabled=self.args.fp16_precision)

        # save config file
        save_config_file(self.writer.log_dir, self.args)

        n_iter = 0
        logging.info(f"Start SimCLR training for {self.args.epochs} epochs.")
        logging.info(f"Training with gpu: {not self.args.disable_cuda}.")

        for epoch_counter in range(1, self.args.epochs + 1):
            for images, _ in tqdm(train_loader):
                images = torch.cat(images, dim=0)

                images = images.to(self.args.device)

                with autocast(enabled=self.args.fp16_precision):
                    features = self.model(images)
                    logits, labels = self.info_nce_loss(features)
                    loss = self.criterion(logits, labels)

                self.optimizer.zero_grad()

                scaler.scale(loss).backward()

                scaler.step(self.optimizer)
                scaler.update()
                if n_iter % self.args.log_every_n_steps == 0:
                    top1, top5 = accuracy(logits, labels, topk=(1, 5))
                    self.writer.add_scalar('loss', loss, global_step=n_iter)
                    self.writer.add_scalar('acc/top1', top1[0], global_step=n_iter)
                    self.writer.add_scalar('acc/top5', top5[0], global_step=n_iter)
                    self.writer.add_scalar('learning_rate', self.scheduler.get_last_lr()[0], global_step=n_iter)

                n_iter += 1

            # warmup for the first 10 epochs
            if epoch_counter > 10:
                self.scheduler.step()
            logging.debug(f"Epoch: {epoch_counter + 1}\tLoss: {loss}\tTop1 accuracy: {top1[0]}")
            
            if epoch_counter % 5 == 0:
            
                checkpoint_name = 'checkpoint_{:04d}.pth'.format(self.args.epochs)
                save_checkpoint({
                    'epoch': self.args.epochs,
                    'arch': self.args.arch,
                    'model': self.model.state_dict(),
                    'optimizer': self.optimizer.state_dict(),
                }, is_best=False, filename=os.path.join(self.writer.log_dir, checkpoint_name))

        logging.info("Training has finished.")
        # save model checkpoints
        checkpoint_name = 'checkpoint_{:04d}.pth'.format(self.args.epochs)
        save_checkpoint({
            'epoch': self.args.epochs,
            'arch': self.args.arch,
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
        }, is_best=False, filename=os.path.join(self.writer.log_dir, checkpoint_name))
        logging.info(f"Model checkpoint and metadata has been saved at {self.writer.log_dir}.")


In [9]:
from model import Simple3DCNN_SimCLR, VoxVGG_SimCLR, VoxResNet_SimCLR

if args.arch == 'simple':
    model = Simple3DCNN_SimCLR(out_dim=args.out_dim)
elif args.arch == 'vgg':
    model = VoxVGG_SimCLR(out_dim=args.out_dim)
elif args.arch == 'resnet':
    model = VoxResNet_SimCLR(out_dim=args.out_dim)
else:
    raise InvalidBackboneError("Invalid backbone architecture.")
print(model)

VoxResNet_SimCLR(
  (backbone): VoxResNet(
    (conv1a): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1a): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1b): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (bn1b): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1c): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
    (voxres2): Sequential(
      (0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU(inplace=True)
      (2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
      (3): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): ReLU(inplace=True)
      (5): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=same)
    )
    (voxres3): Sequential(
      (0): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1)

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=0, last_epoch=-1)

simclr = SimCLR(model=model, optimizer=optimizer, scheduler=scheduler, args=args)
simclr.train(train_loader)

 13%|█▎        | 63/495 [10:09:02<69:36:14, 580.03s/it]  


KeyboardInterrupt: 